In [1]:
import os, pandas, numpy, moment, xmlmanip, csv, datetime, mylittlehelpers, time
from dateutil import relativedelta
os.environ['INTERFACE_CONF_FILE'] = '/home/jjorissen/interface_secrets.conf'
from tangier_api.async_api import AsyncScheduleConnection, HandleResponses
from tangier_api.api import ScheduleConnection, ProviderConnection, LocationConnection
from apprest_interface.api import APPRestConnection
import requests, json

In [2]:
site_id_file = '/home/jjorissen/Documents/TangierDocs/LocationSiteIds.xlsx'
site_ids = sorted(pandas.read_excel(site_id_file).dropna()['Site ID'].unique())

In [3]:
def months_prior(_datetime, months_ago):
    return _datetime - relativedelta.relativedelta(months=months_ago)

def days_later(_datetime, days_later):
    return _datetime + relativedelta.relativedelta(days=days_later)

In [7]:
schedule = ScheduleConnection()
today = datetime.datetime.now()
start = time.time()
schedule_df = pandas.DataFrame()
for i in range(-11, 0, 1):
    x_months_ago = months_prior(today, -i)
    x_plus_one_months_ago_minus_one_day = days_later(months_prior(today, -i-1), -1)
    start_date = x_months_ago.strftime("%Y-%m-%d")
    end_date = x_plus_one_months_ago_minus_one_day.strftime("%Y-%m-%d")
    temp_schedule_list = schedule.get_schedule_values_list(start_date, end_date, site_ids)
    if temp_schedule_list:
        temp_schedule_df = pandas.DataFrame(temp_schedule_list)
        schedule_df = schedule_df.append(temp_schedule_list)
        print(mylittlehelpers.time_elapsed(start), start_date, end_date)

    00:00:44 2016-12-24 2017-01-23
    00:01:14 2017-01-24 2017-02-23
    00:01:43 2017-02-24 2017-03-23
    00:02:14 2017-03-24 2017-04-23
    00:02:46 2017-04-24 2017-05-23
    00:03:19 2017-05-24 2017-06-23
    00:03:51 2017-06-24 2017-07-23
    00:04:41 2017-07-24 2017-08-23
    00:05:37 2017-08-24 2017-09-23
    00:06:33 2017-09-24 2017-10-23


In [10]:
provider_ids = list(schedule_df['empid'].unique())
provider_ids.remove(numpy.nan)
provider = ProviderConnection()
provider_df = pandas.DataFrame(provider.provider_info_values_list(provider_ids))
provider_df_columns = list(provider_df.columns)
provider_df_columns.remove('@action')
provider_df = provider_df[provider_df_columns]

In [3]:
location = LocationConnection()
location_df = pandas.DataFrame(location.location_info_values_list(site_ids=site_ids))
location_df_columns = list(location_df.columns)
location_df_columns.remove('@action')
location_df = location_df[location_df_columns]

In [12]:
schedule_df.to_csv('schedule_flat.csv', quoting=csv.QUOTE_ALL, index=False)
provider_df.to_csv('provider_flat.csv', quoting=csv.QUOTE_ALL, index=False)
location_df.to_csv('location_flat.csv', quoting=csv.QUOTE_ALL, index=False)

In [1]:
location_df

NameError: name 'location_df' is not defined